In [25]:
import numpy as np
import pandas as pd
import keras
from keras import layers, models
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam, Adamax, SGD
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from google.colab import files
import math

# Ladataan data CSV-tiedostosta
file_path = "data_from_mysql_where_g160.csv"
data = pd.read_csv(file_path)

# Tarkastetaan datan rakenne
print(data.head())

# Suodatetaan pois kaikki rivit, joissa 'sensorvalue_d' on 0
data_filtered = data[data['sensorvalue_d'] != 0]

x_data = data_filtered[['sensorvalue_a', 'sensorvalue_b', 'sensorvalue_c']].values  # x, y, z
y_data = data_filtered['sensorvalue_d'].values # suunta

# Skaalaus [0, 1] väliin
def scale_to_unit_interval(data):
    min_vals = np.min(data, axis=0)  # Minimiarvot sarakkeittain
    max_vals = np.max(data, axis=0)  # Maksimiarvot sarakkeittain
    # Varmistetaan, ettei ole nollalla jakamista
    ranges = max_vals - min_vals
    ranges[ranges == 0] = 1  # Jos max == min, käytetään 1:stä

    return (data - min_vals) / ranges  # Skaalaus [0, 1] väliin

# Skaalataan syötteet
x_data_scaled = scale_to_unit_interval(x_data)

# Suuntaa on 6 luokkaa
num_classes = 6
y_data = keras.utils.to_categorical(y_data - 1, num_classes)

x_train, x_test, y_train, y_test = train_test_split(x_data_scaled, y_data, test_size=0.2, random_state=42)

print(f"x_train shape: {x_train.shape}")
print(f"x_test shape: {x_test.shape}")

# Lisätään kohinaa syötteisiin
def add_noise_to_data(data, noise_factor=0.05):
    noise = np.random.normal(0, noise_factor * np.std(data), data.shape)
    return data + noise

# Lisää kohinaa harjoitus- ja testidataan
x_train_noisy = add_noise_to_data(x_train)
x_test_noisy = add_noise_to_data(x_test)

# Määritellään malli
model = keras.Sequential(
    [
        keras.Input(shape=(x_train_noisy.shape[1],)),  # Syötemuoto (x, y, z)
        layers.Dense(256, activation="relu"),  # Ensimmäinen tiheä kerros
        layers.Dropout(0.2),  # Dropout, jotta ylikoulutus ei tapahdu
        layers.Dense(128, activation="relu"),  # Toinen tiheä kerros
        layers.LeakyReLU(negative_slope=0.2),  # Vaihtoehto ReLU:lle
        layers.Dense(num_classes, activation="softmax"),  # Lopullinen luokittelukerros
    ]
)

model.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Koulutetaan mallia
batch_size = 128
epochs = 20

optimizer = Adam(learning_rate=0.001)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

model.fit(x_train_noisy, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[early_stopping])

# Arvioidaan malli testidatalla
score = model.evaluate(x_test_noisy, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

# Tallennetaan malli (painot ja rakenne)
model.save('my_model.keras')

y_pred = model.predict(x_test_noisy)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

print("Classification Report:\n", classification_report(y_test_classes, y_pred_classes))
print("Confusion Matrix:\n", confusion_matrix(y_test_classes, y_pred_classes))
print("")

# Osa 2

# Haetaan mallin painot
weights = model.get_weights()

# Painot ovat lista, jossa on numpy-taulukoita
for idx, weight in enumerate(weights):
    print(f"Painojen ja biasin {idx} muoto: {weight.shape}")
    print("")
    #print(f"Painot ja bias {idx}:", weight)

'''
# Aktivointifunktiot
def relu(x):
    return np.maximum(0, x)  # ReLU-funktio
'''

def relu(x):
    # Luo tyhjä lista tuloksia varten
    output = []

    # Käydään läpi kaikki syötteen arvot
    for val in x:
        # Lisää tulokseen 0, jos arvo on negatiivinen, muuten lisää arvo itsessään
        if val > 0:
            output.append(val)
        else:
            output.append(0)

    return np.array(output)  # Palautetaan lista

# Aktivointifunktio Softmax
def softmax(z):
    exp_values = [2.718 ** i for i in z]  # Eksponenttiarvot
    total = sum(exp_values)  # Lasketaan summan eksponentit
    return [exp_value / total for exp_value in exp_values]  # Jaa eksponenttiarvot kokonaisuudella

# Syöte
for i in range(len(data_filtered)):
    # Haetaan kunkin rivin arvot
    x = data_filtered['sensorvalue_a'].values[i]
    y = data_filtered['sensorvalue_b'].values[i]
    z = data_filtered['sensorvalue_c'].values[i]

    # Muutetaan syöte oikeaan muotoon
    input_data = np.array([x, y, z]).reshape(1, -1)  # Muutetaan muotoon (1, 3)

print("Input data shape:", input_data.shape)

# Skaalataan syöte
input_data_scaled = scale_to_unit_interval(input_data.astype('float32'))

print("Input data scaled shape:", input_data_scaled.shape)

# Oikeat painot ja biasit
weights_0, bias_0 = weights[0], weights[1]
weights_1, bias_1 = weights[2], weights[3]
weights_2, bias_2 = weights[4], weights[5]

#Etenee syötteestä piilokerrosten kautta ulostuloon
def forward_propagation(input_data_scaled):
    # Piilokerros 1
    z0 = []
    for k in range(len(weights_0[0])):  # Käydään läpi piilokerroksen neuronit
        z0_value = sum(input_data_scaled[i] * weights_0[i][k] for i in range(len(input_data_scaled))) + bias_0[k]
        z0.append(z0_value)
    a0 = [relu(z) for z in z0]  # Aktivointi

    # Piilokerros 2
    z1 = []
    for k in range(len(weights_1[0])):
        z1_value = sum(a0[i] * weights_1[i][k] for i in range(len(a0))) + bias_1[k]
        z1.append(z1_value)
    a1 = [relu(z) for z in z1]  # Aktivointi

    # Ulostulokerros
    z2 = []
    for k in range(len(weights_2[0])):  # Kolmannen kerroksen laskentaa (ulostulo)
        z2_value = sum(a1[i] * weights_2[i][k] for i in range(len(a1))) + bias_2[k]
        z2.append(z2_value)
    output = softmax(z2)  # Softmax aktivointi

    # Muotoillaan tulos muotoon (1, 6) ja pyöristetään arvoja samalla tarkkuudella kuin model.predict
    output = np.array(output)  # Varmistetaan, että output on numpy-taulukko
    output = np.round(output, 6).reshape(1, -1)  # Muotoillaan ja pyöristetään

    # Tulostetaan arvot, jotka ovat indekseissä 0, 3, 6, 9, jne.
    selected_values = output[0][::3]  # Valitaan arvot, jotka ovat paikoissa 0, 3, 6, 9 jne.

    return selected_values

# Asetetaan NumPy:n tulostustapa niin, että ei käytetä tieteellistä merkintää
np.set_printoptions(precision=6, suppress=True)

# Lasketaan tulos syötteelle (x, y, z)
result = forward_propagation(input_data_scaled)
print("\nVerkon ulostulo: (forward_propagation):", result)

# Lasketaan ennuste koulutetulla mallilla
prediction = model.predict(input_data_scaled)
print("\nEnnuste (model.predict):", prediction)

# Lasketaan ero tulosten välillä
# Ensimmäinen vaihe on varmistaa, että molemmat result ja prediction ovat numpy-taulukkoja
result = np.array(result).reshape(-1)  # Muutetaan result tasaiseksi taulukoksi
prediction = np.array(prediction).reshape(-1)  # Muutetaan prediction tasaiseksi taulukoksi

# Lasketaan ero
difference = np.abs(result - prediction)  # Lasketaan itseisarvoero

# Tulostetaan ero
print("\nEro (absoluuttinen ero result ja prediction välillä):", difference)

# Keskimääräinen ero
mean_difference = np.mean(difference)
print("\nKeskimääräinen ero: ", mean_difference)

# Tallennetaan painot ja biasit header-tiedostoon
header_file = "neuroverkonKertoimet.h"

with open(header_file, "w") as f:
    f.write("#ifndef NEUROVERKONKERTOIMET_H\n")
    f.write("#define NEUROVERKONKERTOIMET_H\n\n")

    # Kirjoitetaan painot ja biasit jokaiselle kerrokselle
    for idx, weight in enumerate(weights):
        if len(weight.shape) == 2:  # Painot (matriisi)
            f.write(f"float weights_{idx}[{weight.shape[0]}][{weight.shape[1]}] = {{\n")
            for row in weight:
                f.write("    {" + ", ".join(map(str, row)) + "},\n")
            f.write("};\n\n")
        elif len(weight.shape) == 1:  # Bias (vektori)
            f.write(f"float biases_{idx}[{weight.shape[0]}] = {{")
            f.write(", ".join(map(str, weight)))
            f.write("};\n\n")

    f.write("#endif // NEUROVERKONKERTOIMET_H\n")

     id            timestamp  groupid  from_mac  to_mac  sensorvalue_a  \
0  8125  2024-12-02 10:10:47      160         0       0         1507.0   
1  8126  2024-12-02 10:10:47      160         0       0         1518.0   
2  8127  2024-12-02 10:10:48      160         0       0         1518.0   
3  8128  2024-12-02 10:10:48      160         0       0         1511.0   
4  8129  2024-12-02 10:10:49      160         0       0         1512.0   

   sensorvalue_b  sensorvalue_c  sensorvalue_d  sensorvalue_e  sensorvalue_f  
0         1387.0         1807.0            1.0            0.0              0  
1         1393.0         1806.0            1.0            0.0              0  
2         1384.0         1806.0            1.0            0.0              0  
3         1389.0         1804.0            1.0            0.0              0  
4         1393.0         1803.0            1.0            0.0              0  
x_train shape: (1060, 3)
x_test shape: (265, 3)


Model: "sequential_24"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_74 (Dense)                     │ (None, 256)                 │           1,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_24 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_75 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_24 (LeakyReLU)           │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_76 (Dense)                     │ (None, 6)                   │             774 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 34,694 (135.52 KB)

 Trainable params: 34,694 (135.52 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 217ms/step - accuracy: 0.3370 - loss: 1.7556 - val_accuracy: 0.5755 - val_loss: 1.6118
Epoch 2/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5700 - loss: 1.5721 - val_accuracy: 0.7925 - val_loss: 1.3998
Epoch 3/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8719 - loss: 1.3605 - val_accuracy: 1.0000 - val_loss: 1.1389
Epoch 4/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 1.1028 - val_accuracy: 1.0000 - val_loss: 0.8543
Epoch 5/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.8132 - val_accuracy: 1.0000 - val_loss: 0.5950
Epoch 6/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.5604 - val_accuracy: 1.0000 - val_loss: 0.3853
Epoch 7/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.3674 - val_accuracy: 1.0000 - val_loss: 0.2440
Epoch 8/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.2356 - val_accuracy: 1.0000 - val_loss: 0.1515
Epoch 